In [ ]:
# Import library

# Packages required
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits
import os
from deconvbench import Deconvbench

from amiral import instructment, utils, parameter, gradient, minimisation, array, plotting


# from plotting import plot_PSF_PSD as amiral_plt
from scipy.optimize import minimize 

%matplotlib inline

import tools

rcParams["figure.figsize"] = 20,33

In [ ]:
# Function for getting snr
def get_snr (array, noise):
    
    mean = np.mean(array)
    sig2 = np.std(noise)
    
    snr = mean / sig2
    
    return snr

def psd_object (param):
    
    rho = np.fft.fftshift(utils.dist(768))/param[1]
    psd_obj =  param[0]/ (np.power(rho,param[2]) + 1.)
    
    return psd_obj
    
def plot_psd_object(psd_obj): 
    
    fig, ax = plt.subplots()
    ycent = int((256*aosys_cls.samp_factor[0])//2)

    ax.plot(np.abs(psd_obj[ycent,...]))
    ax.set_title('PSD Object(total)')
    ax.axhline(y=1, color = 'r', ls = '--')
    
    pass

def create_psfao19_otf (otf_tel, guess, aosys_cls): 
    
    # Use PSFAO19 model to create a PSF
    psd_ao = aosys_cls.psd_residual_ao (guess = guess)
    psd_halo = aosys_cls.psd_residual_halo(r0 = guess[0])
    
    psd = psd_ao + psd_halo

    otf_atmo = aosys_cls.otf_atmo(psd)
    otf_total = otf_atmo*otf_tel
      
    return otf_atmo,otf_total

In [ ]:
# PATH

wdir = "/Users/alau/Data/amiral_fits/VESTA/SNR/"
path_asteriod = '/Users/alau/IDLWorkspace/Data/Vesta_OASIS/2018-06-08T07_58_51.472.fits'

# Parameters for the PSF
psf_dict = {
    "r0": 0.15,                  
    "background": 0.,      
    "amplitude": 0.5,       
    "ax": 0.05,                            
    "beta": 1.5, 
    "mu": 0., 
    "rho0": 0., 
    "p": 0. 
}

# Variable
RON = 10. # CCD read-out noise standard-deviation [e-]
FLUX = np.linspace(5e6,5e9,10)
DIMENSION = int(256*3)

flux_snr = FLUX[9]
print(flux_snr)

# psf_dict["r0"] = 0.1
# psf_dict["amplitude"] = 0.5

Get the object

In [ ]:
data = fits.open(path_asteriod)
asteriod = data[0].data
asteriod_resize = np.zeros((DIMENSION,DIMENSION))


cuta = DIMENSION//2-128
cutb = DIMENSION//2+128
 
asteriod_resize[cuta:cutb,cuta:cutb] = asteriod
_asteriod_resize = asteriod_resize/np.sum(asteriod_resize)

# Calibrating the flux
asteriod_resize = asteriod_resize/np.sum(asteriod_resize)*flux_snr

Add Photon noise

In [ ]:
rng = np.random.default_rng()
photon_noise = rng.poisson(asteriod_resize)
print("\nPhoton Noise of the object: ", np.sum((photon_noise)))

ft_photon_noise = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(photon_noise)))
ft_obj = np.fft.fft2(np.fft.ifftshift(asteriod_resize))

We need a PSF

In [ ]:
# Set up the telescope and produce a PSF
amiral_dict = {
    "r0": 0.2,                  
    "background": 0.,      
    "amplitude": 0.5,       
    "ax": 0.05,                            
    "beta": 1.5, 
    "mu": 0., 
    "rho0": 0., 
    "p": 0. 
}

aosys_dict = {
    'diameter': 8 , 
    'occ_ratio': 0.14 , 
    'no_acutuator' : 20, 
    'wavelength': 500, 
    'dimension': 256,
    'resolution_rad' : 2.083e-8
}

# Passing parametpsd_arrayers from the telesope to the aosystem
aosys_cls = instructment.aoSystem( 
        diameter = aosys_dict['diameter'], occ_ratio = aosys_dict['occ_ratio'], 
        no_acutuator= aosys_dict['no_acutuator'], wavelength = aosys_dict['wavelength']*1e-9, 
        resolution_rad = aosys_dict['resolution_rad'], 
        dimension=aosys_dict['dimension'])  


# # Parameters for the PSF
# psf_dict = {
#     "r0": 0.1,                  
#     "background": 0.0,      
#     "amplitude": 0.5,       
#     "ax": 0.05,                            
#     "beta": 1.5, 
#     "mu": 0., 
#     "rho0": 0., 
#     "p": 0. 
# }

psf_keys, psf_param = utils.dict2array(psf_dict)
amiral_keys, psf_guess = utils.dict2array(amiral_dict)

In [ ]:
psd_ao = aosys_cls.psd_residual_ao (psf_param)
psd_halo = aosys_cls.psd_residual_halo(psf_dict['r0'])

psd = psd_halo + psd_ao 

pupil = aosys_cls.get_pupil_plane()
otf_tel = aosys_cls.pupil_to_otf_tel(pupil)

integral, SR = aosys_cls.psd_integral(psd_ao, psf_dict['r0'])

otf_atmo = aosys_cls.otf_atmo(psd)

otf_total = np.fft.ifftshift(otf_atmo * otf_tel)

psf_total = np.fft.ifft2(otf_total)

psf_tel = np.fft.ifft2(otf_tel)

plt.imshow(np.real(np.log10(otf_total)))

Get Gaussian Noise

In [ ]:
ft_image = ft_obj* otf_total

plt.imshow(np.log10(np.real(ft_image)))

In [ ]:
rng = np.random.default_rng()
_obj = np.real(np.fft.ifft2(ft_image))

photon_noise = rng.poisson(_obj)

print("\nPhoton Noise of the object: ", np.sum((photon_noise)))

gauss_noise = np.random.randn(DIMENSION,DIMENSION)*RON
noise = photon_noise + gauss_noise

empty_arr = np.zeros((DIMENSION,DIMENSION))+1.

psd_noise = np.abs(np.fft.fftshift(np.fft.fft2(noise)))**2
psd_asteroid = np.abs(np.fft.fftshift(ft_image))**2

snr = psd_asteroid/psd_noise

fig, ax = plt.subplots(1,1)

rcParams['figure.figsize'] = 33 ,24

im = ax.imshow(np.log10(snr))
fig.colorbar(im, ax=ax)
ax.set_title('Object')


est_obj = tools.wiener_filter(ft_image,otf_total,snr)


obs_image = noise + _obj


In [ ]:
fig, ax = plt.subplots(1,4)

rcParams['figure.figsize'] = 33 ,24

im = ax[0].imshow(asteriod_resize)
fig.colorbar(im, ax=ax[0])
ax[0].set_title('Object')

im1 = ax[1].imshow(obs_image)
fig.colorbar(im1, ax=ax[1])
ax[1].set_title('Observed Image')

im2 = ax[2].imshow(asteriod_resize-obs_image)
fig.colorbar(im2, ax=ax[2])
ax[2].set_title('Noise')


im3 = ax[3].imshow(asteriod_resize+noise)
fig.colorbar(im3, ax=ax[3])
ax[3].set_title('Obj + Noise')

In [ ]:
print("\nFlux of the object: ", np.sum(asteriod_resize))
print("\nSNR: ", np.sum(asteriod_resize)/np.sum(noise))

In [ ]:
print("Sum: ", np.sum(obs_image))
print("\nNoise of the object: ", np.sum((noise)))
print("\nPhoton Noise of the object: ", np.sum((photon_noise)))
print("Retrieved Flux : ",np.sum(obs_image) - (np.sum(noise)))

In [ ]:
img = obs_image

# _img = fits.open("/Users/alau/Data/amiral_fits/VESTA/SNR/2021mar29/case_8/2018-06-08T05_27_05.809_1.fits")
# img = _img[0].data

# _noise = fits.open("/Users/alau/Data/amiral_fits/VESTA/SNR/2021mar29/case_8/2018-06-08T05_27_05.809_noise_1.fits")
# noise = _noise[0].data

amiralparam = parameter.amiralParam(img ,guess = psf_guess, aosys = aosys_cls)

# k_hat = (1e6**2)
# psd_noise = np.sum(np.abs(noise)**2)

# mu = psd_noise / k_hat

# print(mu)

In [ ]:
# What variables to be minimised
param_mask = np.asarray([1,0,0,0,0])
hyper_param_mask = np.asarray([0,0,0])

mask = np.concatenate((param_mask,hyper_param_mask))

hyper_guess = amiralparam.hyperparam_initial(psf_guess)
hyper_min, hyper_max = amiralparam.hyperparam_bound(psf_guess, p_upperbound = 100.)

psf_guess[-3] = hyper_guess[0] 
psf_guess[-2] = hyper_guess[1] 
psf_guess[-1] = hyper_guess[2] 


# psf_guess[-3] = mu
# psf_guess[-1] = 2.91
# psf_guess[-2] = 1.32*()


# psf_guess[-3] = 2.01197235e-08 
# psf_guess[-2] = 0.82*(DIMENSION)/512
# psf_guess[-1] = 2.91

param_min = np.asarray([0.05,0,0,1e-8,1.01])
param_max =  np.asarray([1.,1e8,1e8,1e3,10])

upperbound = np.concatenate((param_max, hyper_max))
lowerbound = np.concatenate((param_min, hyper_min))

param_numerical_condition = np.array([1., 1e-4, 1., 1., 1.])
hyperparam_numerical_condition = np.array([hyper_guess[0], hyper_guess[1], 1.])

numerical_condition = np.concatenate((param_numerical_condition, hyperparam_numerical_condition))

amiral_cls = parameter.amiral(img=img, guess=psf_guess, aosys = aosys_cls, upperbound = upperbound, lowerbound= lowerbound, numerical_condition = numerical_condition, fourier_variable = amiralparam.fourier_variable, mask = mask)

In [ ]:
est_criterion, value_criterion, value_grad = amiral_cls.minimisation(psf_guess)

In [ ]:
print(est_criterion)

In [ ]:
est_otf_atmo, est_otf = create_psfao19_otf(otf_tel, est_criterion[0:5], aosys_cls)
est_psf = np.fft.ifft2(est_otf)

In [ ]:
est_SR = np.max(np.real(est_psf)) / np.max(np.real(psf_tel))
SR = np.max(psf_total) / np.max(psf_tel)

In [ ]:
np.real(est_SR-SR)*100

In [ ]:
est_SR

In [ ]:
np.max(np.real(est_psf)) / np.max(np.real(psf_tel))

In [ ]:
np.max(psf_total) / np.max(psf_tel)

In [ ]:
np.max(np.real(est_psf))

Intake the estimated criterion for plotting the PSF

In [ ]:
est_param = est_criterion[0:5]
est_otf_atmo, est_otf_total = create_psfao19_otf(otf_tel,est_param,aosys_cls)

In [ ]:
est_psf = np.fft.ifft2(est_otf_total)
plt.imshow(np.real(np.log10(est_psf-psf_total)))

In [ ]:
dec = Deconvbench(obs_image,est_psf,ron = 10)
dec.verbose_modulo = 10 # print every 10 iteration
dec.regularization.scale *= 4. # sharpen details (reduce regularization)
estim = dec.run()

In [ ]:
estim_norm = estim/asteriod_resize.max()
asteriod_resize_norm = asteriod_resize/asteriod_resize.max()


fig, ax = plt.subplots(1,2)

rcParams['figure.figsize'] = 33 ,24

im = ax[0].imshow(np.abs(estim_norm-asteriod_resize_norm))
fig.colorbar(im, ax=ax[0])
ax[0].set_title('Residual (Normalised)\nDiff(SR) = 2.16%')

im1 = ax[1].imshow(np.abs(asteriod_resize-estim))
fig.colorbar(im1, ax=ax[1])
ax[1].set_title('Residual\nDiff(SR) = 2.16%')

In [ ]:
dec = Deconvbench(obs_image,psf_total,ron = 10)
dec.verbose_modulo = 10 # print every 10 iteration
dec.regularization.scale *= 4. # sharpen details (reduce regularization)
estim_1 = dec.run()

In [ ]:
perfect_deconv = estim_1
diff = estim_1-asteriod_resize

fig, ax = plt.subplots(1,3)

rcParams['figure.figsize'] = 33 ,24

im = ax[0].imshow(asteriod_resize)
fig.colorbar(im, ax=ax[0])
ax[0].set_title('Object')

im1 = ax[1].imshow(obs_image)
fig.colorbar(im1, ax=ax[1])
ax[1].set_title('Observed Image')

im2 = ax[2].imshow(diff)
fig.colorbar(im2, ax=ax[2])
ax[2].set_title('Residual\nDiff(SR) = 2.16%')


In [ ]:
estim_1_norm = estim_1/asteriod_resize.max()
asteriod_resize_norm = asteriod_resize/asteriod_resize.max()

fig, ax = plt.subplots()

rcParams['figure.figsize'] = 33 ,24

im = ax.imshow(np.abs(estim_1_norm-asteriod_resize_norm))
fig.colorbar(im, ax=ax)
ax.set_title('Exact PSF Residual(Normalised)\nDiff(SR) = 2.16%')



In [ ]:
fig, ax = plt.subplots()

rcParams['figure.figsize'] = 33 ,24

im = ax.imshow(asteriod_resize)
fig.colorbar(im, ax=ax)
ax.set_title('Object')

In [ ]:
from deconvbench.stat import DSPFit, Circmoyto2D
from deconvbench import RegulPSD

rho, psd_param, _, psd1d = DSPFit(obs_image)
psd1 = Circmoyto2D(rho,psd1d,obs_image.shape[0])


#%% ITERATIVE DECONVOLUTION
dec_psd1 = Deconvbench(obs_image, est_psf, ron=10, positivity=False, verbose=True)
dec_psd1.verbose_modulo = 50
dec_psd1.regularization = RegulPSD(psd1) # set PSD regularization
estD1 = dec_psd1.run()

In [ ]:
fig, ax = plt.subplots()

rcParams['figure.figsize'] = 33 ,24

im = ax.imshow(estD1)
fig.colorbar(im, ax=ax)
ax.set_title('Deconv')

In [ ]:
psd_param

In [ ]:
from deconvbench.stat import DSPFit, Circmoyto2D
from deconvbench import RegulPSD

rho, psd_param, _, psd1d = DSPFit(obs_image)
psd1 = Circmoyto2D(rho,psd1d,obs_image.shape[0])


#%% ITERATIVE DECONVOLUTION
dec_psd = Deconvbench(obs_image, psf_total, ron=10, positivity=False, verbose=True)
dec_psd.verbose_modulo = 50
dec_psd.regularization = RegulPSD(psd1) # set PSD regularization
estD = dec_psd.run()

In [ ]:
fig, ax = plt.subplots()

rcParams['figure.figsize'] = 33 ,24

im = ax.imshow(estD)
fig.colorbar(im, ax=ax)
ax.set_title('Deconv')

print(np.sum(estD))